# Импортирање на потребните библиотеки

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.8 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 53.2 MB/s 
     |████████████████████████████████| 3.3 MB 39.2 MB/s 
     |████████████████████████████████| 50 kB 7.4 MB/s 
     |████████████████████████████████| 895 kB 48.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 5.3 MB/s 
     |████████████████████████████████| 243 kB 37.6 MB/s 
     |████████████████████████████████| 119 kB 51.2 MB/s 


In [ ]:
from transformers import BertTokenizerFast, TFBertModel
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error
from datasets import load_dataset
import pandas as pd
import numpy as np

# Вчитување податоци

In [ ]:
# го вчитуваме податочното множество SQuAD кое е составено од прашања
# контекст според кој треба да се одговорат прашањата и одговори
data = load_dataset('squad')

# за полесно користење на податоците, ги трансформираме
# во pandas DataFrame објекти
train = pd.DataFrame().from_dict(data['train'])
test = pd.DataFrame().from_dict(data['validation'])

# за побрзо извршување на показните примери, користиме само дел
# од податочното множество
train = train[:2000]
test = test[:1000]

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [ ]:
# приказ на дел од записите во податочното множество
train.head()

,id,title,context,question,answers
0,5733be284776f41900661182,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,"{'text': ['Saint Bernadette Soubirous'], 'answ..."
1,5733be284776f4190066117f,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,"{'text': ['a copper statue of Christ'], 'answe..."
2,5733be284776f41900661180,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,"{'text': ['the Main Building'], 'answer_start'..."
3,5733be284776f41900661181,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,{'text': ['a Marian place of prayer and reflec...
4,5733be284776f4190066117e,University_of_Notre_Dame,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,{'text': ['a golden statue of the Virgin Mary'...


In [ ]:
# за полесно користење на податоците, ги сместуваме во посебни низи
# прашањата, одговорите и контекстот според кој се одговараат прашањата
train_contexts = train['context'].values.tolist()
train_questions = train['question'].values.tolist()
train_answers = train['answers'].values.tolist()

# за потребите на моделот кој ќе се тренира, за секој одговор е потребно
# да го знаеме индексот на почетниот и крајниот знак
# бидејќи во податочното множество е достапен само индексот на почетниот знак
# потребно е да го пресметаме индексот на крајниот знак 
for answer, context in zip(train_answers, train_contexts):
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = start_idx + len(gold_text)

    answer['text'] = gold_text

    # понекогаш индексот на почетниот знак може да се разликува за 1 или 2
    # позиции од вистинската и во таквите случаи потребно е да промениме
    # индексот на почетниот знак
    if context[start_idx:end_idx] == gold_text:
        answer['answer_start'] = start_idx
        answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1 
    elif context[start_idx-2:end_idx-2] == gold_text:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

In [ ]:
# за полесно користење на податоците, ги сместуваме во посебни низи
# прашањата, одговорите и контекстот според кој се одговараат прашањата
test_contexts = test['context'].values.tolist()
test_questions = test['question'].values.tolist()
test_answers = test['answers'].values.tolist()

# за потребите на моделот кој ќе се тренира, за секој одговор е потребно
# да го знаеме индексот на почетниот и крајниот знак
# бидејќи во податочното множество е достапен само индексот на почетниот знак
# потребно е да го пресметаме индексот на крајниот знак 
for answer, context in zip(test_answers, test_contexts):
    gold_text = answer['text'][0]
    start_idx = answer['answer_start'][0]
    end_idx = start_idx + len(gold_text)

    answer['text'] = gold_text

    # понекогаш индексот на почетниот знак може да се разликува за 1 или 2
    # позиции од вистинската и во таквите случаи потребно е да промениме
    # индексот на почетниот знак
    if context[start_idx:end_idx] == gold_text:
        answer['answer_start'] = start_idx
        answer['answer_end'] = end_idx
    elif context[start_idx-1:end_idx-1] == gold_text:
        answer['answer_start'] = start_idx - 1
        answer['answer_end'] = end_idx - 1 
    elif context[start_idx-2:end_idx-2] == gold_text:
        answer['answer_start'] = start_idx - 2
        answer['answer_end'] = end_idx - 2

# BERT

## Претпроцесирање на податоци

In [ ]:
# за трансформација на текстуалните податоци во формат соодветен за моделот
# BERT ќе користиме BertTokenizerFast
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# секој запис се трансформира во потребниот формат
# со аргументот max_length се дефинира максималната должина
# со аргументот pad_to_max_length се означува дека ако записот е пократок од
# максималната должина, тогаш се дополнува со токени до посакуваната должина
# со аргументот truncation се означува дека ако записот е подолг од
# максималната должина, тогаш се отстрануваат токени се додека не ја 
# добие посакуваната должина
train_encodings = bert_tokenizer(train_contexts, train_questions, max_length=100, truncation=True, pad_to_max_length=True)
train_input_ids = train_encodings['input_ids']
train_attention_masks = train_encodings['attention_mask']

# дефинираме празни листи за почетните и крајните позиции на одговорите
train_start_positions, train_end_positions = [], []
for i in range(len(train_answers)):
    # секоја почетна и крајна позиција се трансформира индексот на токенот
    # на соодветната позиција
    train_start_positions.append(train_encodings.char_to_token(i, train_answers[i]['answer_start']))
    train_end_positions.append(train_encodings.char_to_token(i, train_answers[i]['answer_end'] - 1))

    # доколку некоја почетна или крајна позиција е поголема од максималната
    # должина, тогаш вредноста се поставува на максималната должина
    if train_start_positions[-1] is None:
        train_start_positions[-1] = bert_tokenizer.model_max_length
    if train_end_positions[-1] is None:
        train_end_positions[-1] = bert_tokenizer.model_max_length

In [ ]:
# секој запис се трансформира во потребниот формат
# со аргументот max_length се дефинира максималната должина
# со аргументот pad_to_max_length се означува дека ако записот е пократок од
# максималната должина, тогаш се дополнува со токени до посакуваната должина
# со аргументот truncation се означува дека ако записот е подолг од
# максималната должина, тогаш се отстрануваат токени се додека не ја 
# добие посакуваната должина
test_encodings = bert_tokenizer(test_contexts, test_questions, max_length=100, truncation=True, pad_to_max_length=True)
test_input_ids = test_encodings['input_ids']
test_attention_masks = test_encodings['attention_mask']

# дефинираме празни листи за почетните и крајните позиции на одговорите
test_start_positions, test_end_positions = [], []
for i in range(len(test_answers)):
    # секоја почетна и крајна позиција се трансформира во индексот 
    # на токенот на соодветната позиција
    test_start_positions.append(test_encodings.char_to_token(i, test_answers[i]['answer_start']))
    test_end_positions.append(test_encodings.char_to_token(i, test_answers[i]['answer_end'] - 1))

    # доколку некоја почетна или крајна позиција е поголема од максималната
    # должина, тогаш вредноста се поставува на максималната должина
    if test_start_positions[-1] is None:
        test_start_positions[-1] = bert_tokenizer.model_max_length
    if test_end_positions[-1] is None:
        test_end_positions[-1] = bert_tokenizer.model_max_length

## Креирање модели

In [ ]:
# за проблемот на одговарање прашања инстанцираме објект од генеричката
# класа за моделот BERT
# при инстанцирање на моделот се дефинира која верзија на моделот BERT ќе се
# користи (bert-base-uncased - верзија на BERT моделот со 12 слоеви тренирана 
# без правење разлика помеѓу мали и големи букви)
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# влезни слоеви со димензија (100,) и тип int32 за индексите 
# на зборови и маските на внимание
input_ids = Input(shape=(100,), name='input_token', dtype='int32')
att_masks = Input(shape=(100,), name='masked_token', dtype='int32')

# спојување на влезните слоеви со слоевите од моделот BERT
bert_in = bert_model(input_ids, attention_mask=att_masks)[1]

# излезни слоеви за индексите на почетните и крајните позиции на одговорот
start = Dense(1, activation='relu', name='start')(bert_in)
end = Dense(1, activation='relu', name='end')(bert_in)

# креирање објект кој ќе го претставува моделот со соодветните 
# влезни и излезни слоеви
bert_qa_model = Model(inputs=[input_ids, att_masks], outputs=[start, end])

# пред да се тренира, потребно е моделот да се компајлира
# со поставување на аргументот loss дефинираме средна квадратна грешка
# како функција на загуба
# со поставување на аргументот optimizer дефинираме Adam оптимизатор со рата
# на учење еднаква на 0.01
# со поставување на аргументот metrics дефинираме точност како метрика за 
# следење на перформансите на моделот при тренирање
bert_qa_model.compile(optimizer=Adam(learning_rate=0.01), loss=mean_squared_error, metrics=['accuracy'])

## Тренирање и евалуација

In [ ]:
# при тренирање на моделот покрај влезните и излезните податоци, потребно е
# да се постави вредност за аргументот epochs што претставува број на епохи
bert_qa_model.fit([np.array(train_input_ids), np.array(train_attention_masks)], 
                  [np.array(train_start_positions), np.array(train_end_positions)], 
                  epochs=3)

Epoch 1/3
63/63 [==============================] - 94s 1s/step - loss: 117737.6328 - start_loss: 57631.8828 - end_loss: 60105.7695 - start_accuracy: 0.0120 - end_accuracy: 0.0030
Epoch 2/3
63/63 [==============================] - 75s 1s/step - loss: 102964.2734 - start_loss: 51014.8359 - end_loss: 51949.4492 - start_accuracy: 0.0120 - end_accuracy: 0.0030
Epoch 3/3
63/63 [==============================] - 75s 1s/step - loss: 102989.8984 - start_loss: 51037.4219 - end_loss: 51952.4766 - start_accuracy: 0.0120 - end_accuracy: 0.0030


In [ ]:
# евалуација на моделот со што се добиваат вредности за
# функцијата на загуба и точноста
bert_qa_model.evaluate([np.array(test_input_ids), np.array(test_attention_masks)],
                       [np.array(test_start_positions), np.array(test_end_positions)])

32/32 [==============================] - 16s 389ms/step - loss: 84015.9922 - start_loss: 41453.2344 - end_loss: 42562.7578 - start_accuracy: 0.0210 - end_accuracy: 0.0110


[84015.9921875,
 41453.234375,
 42562.7578125,
 0.020999999716877937,
 0.010999999940395355]